# Соревнование на Kaggle: teta_ml_1_2025  
## Предсказание fraud по карточной транзакции  

#### Overview  
Соревнование в рамках курса от МТС. Задача: детекция фрода карточных транзакций  

#### Description  
Соревнование по задаче бинарной классификации  

#### Evaluation  
Оценка метрикой F1 score  

### Dataset Description  

#### Files  
train.csv - the training set  
test.csv - the test set  
sample_submission.csv - a sample submission file in the correct format  

#### Columns  
| Column                  | Description |
|-------------------------|-------------|
| transaction_time        | дата и время совершения транзакции |
| merch                  | название или идентификатор продавца или торговой точки, где была совершена транзакция |
| cat_id                 | идентификатор категории товара или услуги, к которой относится транзакция |
| amount                 | сумма транзакции |
| name_1, name_2         | имена, связанные с транзакцией |
| gender                 | пол клиента |
| street                 | название улицы |
| one_city               | город |
| us_state               | штат США |
| post_code              | почтовый индекс |
| lat, lon               | широта и долгота |
| population_city        | численность населения города проживания клиента |
| jobs                   | уровень занятости |
| merchant_lat, merchant_lon | широта и долгота местоположения продавца или точки продажи |
| target                 | целевая переменная |



## Intro
### Описание финального решения:
1) **как обработали данные**;
   
   - Заполнил пропуски - медианным значением для непрерывных переменных и значениями между значениями слева и справа для дат;
   - Категориальные признаки закодировал с помощью CatBoost Encoder (Target Encoding);
   - Скалирование непрерывных переменных не показало достаточного прироста в качестве скора - скорее, модель наоборот выдавала более смещенные предикты, чем ожидалось.
   - Сгенерировал достаточно много собственных признаков (смотреть ноутбук далее);

3) **какой алгоритм вы выбрали**;
   
   - Использовал затюненное на трешхолд дерево решений из семинарских материалов как BaseLine модель;
   - CatBoost для задачи классификации - Final модель.

5) **какие особенности использовали при его обучении**;
   
   - Подбор оптимального трешхолда по обучении модели с целью максимизации оцениваемой в соревновании метрики f1-score;
   - Пробовал передавать категориальные признаки как в сам алгоритм, так и кодировать отдельно - кодировка с помощью CatBoost Encoding отдельно показала более хорошие результаты.

7) **как проводили валидацию алгоритма**.
   
   - Работал со стандартным сплитом выборки на test и train (0.25/0.75)
   - Использовал Optuna для подбора оптимальных гиперпараметров; тем не менее, решение без подбора оказалось лучше по пересчету метрик на Kaggle.

## Outro
### Описание вариантов по дальнейшему улучшению итогового подхода:

1) **идея №1, которая может улучшить метрику, как ее реализовать (что для этого нужно сделать)**;
   
   - **Feature Engeneering**
   - Больше времени уделить на feature engeneering - как показала практика, именно FE дает крайне значимый прирост в значениях метрики;
   - Можно достаточно плотно поработать с координатными признаками;
   - Можно много времени уделить на агрерированные показатели по категориям;
   - Можно пробовать использовать нелинейные трансформации над признаками.

2) **идея №2, которая может улучшить метрику, как ее реализовать (что для этого нужно сделать)**;
   
   - **Качественный Feature Selection**
   - Провести Feature Selection с помощью Permutation Importance и исключить признаки, которые негативно или нейтрально влияют на модель (реализовано по втором соревновании, здесь сугубо выборочно, исходя из ситуации и базового FE);
   - Как показывает практика, множество признаков не всего есть хорошо - что и было выявлено во втором соревновании, где огромное число признаков для модели вносило лишь смещение по RMSE-скору.

     
3) **идея №3, которая может улучшить метрику, как ее реализовать (что для этого нужно сделать)**.
   
   - **Обеспечить робастность скора по f1-score**
   - Лучше проработать подбор трешхолда, а также протестировать модель на более качественной кросс-валидации;
   - StratifiedKFold на 5-10 фолдов и оценка скора как усредненное значение - кажется, достаточно релеватный подход;
   - Adversarial Validation / Kolmogorov-Smirnov Test - оценка различий в распределениях, так как продакшн-ready модель должна иметь четкий мониторинг по этой части;
   - Подумать над Out-of-Time Sample

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import zscore

# pip install category-encoders
import category_encoders as ce

from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import optuna
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score

In [4]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

## Feature Engeneering

### Разберемся с transaction_time

In [7]:
train.isna().mean(), test.isna().mean()

(transaction_time    0.0
 merch               0.0
 cat_id              0.0
 amount              0.0
 name_1              0.0
 name_2              0.0
 gender              0.0
 street              0.0
 one_city            0.0
 us_state            0.0
 post_code           0.0
 lat                 0.0
 lon                 0.0
 population_city     0.0
 jobs                0.0
 merchant_lat        0.0
 merchant_lon        0.0
 target              0.0
 dtype: float64,
 transaction_time    0.0
 merch               0.0
 cat_id              0.0
 amount              0.0
 name_1              0.0
 name_2              0.0
 gender              0.0
 street              0.0
 one_city            0.0
 us_state            0.0
 post_code           0.0
 lat                 0.0
 lon                 0.0
 population_city     0.0
 jobs                0.0
 merchant_lat        0.0
 merchant_lon        0.0
 dtype: float64)

In [8]:
np.sort(train['transaction_time'])

array(['2019-01-01 00:00', '2019-01-01 00:03', '2019-01-01 00:04', ...,
       '2020-03-10 16:07', '2020-03-10 16:07', '2020-03-10 16:08'],
      dtype=object)

#### Перевод в число и дату

In [10]:
train['transaction_time'] = pd.to_datetime(train['transaction_time'])
test['transaction_time'] = pd.to_datetime(test['transaction_time'])

train['transaction_date'] = train['transaction_time'].dt.date
test['transaction_date'] = test['transaction_time'].dt.date

#### Вытащим фичи из дат

In [12]:
import holidays

us_holidays = holidays.US(years=2019)  # Здесь задаю годы, для которых нужны праздники - 
                                        # ошибся с годом, только позже заметил это, надо не только за 2019 брать

for date, name in sorted(us_holidays.items()):
    print(date, name)

2019-01-01 New Year's Day
2019-01-21 Martin Luther King Jr. Day
2019-02-18 Washington's Birthday
2019-05-27 Memorial Day
2019-07-04 Independence Day
2019-09-02 Labor Day
2019-10-14 Columbus Day
2019-11-11 Veterans Day
2019-11-28 Thanksgiving Day
2019-12-25 Christmas Day


In [13]:
# Создаём список праздников для США в 2019 году - СНОВА ОШИБКА
us_holidays = holidays.US(years=[2019])

# Функция для получения названия праздника
def get_holiday_name(date):
    return us_holidays.get(date) if date in us_holidays else "No Holiday"

# Функция для биннинга времени суток
def assign_time_of_day(hour):
    if 6 <= hour < 12:
        return "morning" 
    elif 12 <= hour < 18:
        return "afternoon"
    elif 18 <= hour < 24:
        return "evening" 
    else:
        return "night"

# Обработка train
train['transaction_time_month'] = train['transaction_time'].dt.month
train['transaction_time_week'] = train['transaction_time'].dt.isocalendar().week
train['transaction_time_day_of_the_week'] = train['transaction_time'].dt.dayofweek
train['transaction_time_hour'] = train['transaction_time'].dt.hour
train['transaction_time_minute'] = train['transaction_time'].dt.minute

train["transaction_time_holidays"] = train["transaction_time"].apply(get_holiday_name)
train['transaction_time_binning_by_part'] = train['transaction_time_hour'].apply(assign_time_of_day)

# Обработка test
test['transaction_time_month'] = test['transaction_time'].dt.month
test['transaction_time_week'] = test['transaction_time'].dt.isocalendar().week
test['transaction_time_day_of_the_week'] = test['transaction_time'].dt.dayofweek
test['transaction_time_hour'] = test['transaction_time'].dt.hour
test['transaction_time_minute'] = test['transaction_time'].dt.minute

test["transaction_time_holidays"] = test["transaction_time"].apply(get_holiday_name)
test['transaction_time_binning_by_part'] = test['transaction_time_hour'].apply(assign_time_of_day)

# NICE

## Снова работаем с расстояниями

In [15]:
import numpy as np
from math import atan2, cos, radians, sin, sqrt


def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float, n_digits: int = 0) -> float:
    """
        Функция для расчёта расстояния от точки А до Б по прямой

        :param lat1: Широта точки А
        :param lon1: Долгота точки А
        :param lat2: Широта точки Б
        :param lon2: Долгота точки Б
        :param n_digits: Округляем полученный ответ до n знака после запятой
        :return: Дистанция по прямой с точностью до n_digits
    """

    lat1, lon1, lat2, lon2 = round(lat1, 6), round(lon1, 6), round(lat2, 6), round(lon2, 6)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)

    dlambda = np.radians(lon2 - lon1)
    a = np.sin(dphi / 2) ** 2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda / 2) ** 2

    return round(2 * 6372800 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)), n_digits)  # метры.сантиметры


def bearing_degree(lat1: float, lon1: float, lat2: float, lon2: float, n_digits: int = 0) -> float:
    """
        Функция для расчёта угла между прямой [((lat1, lon1), (lat2, lon2)), (нулевой мередиан)]

        :param lat1: Широта точки А
        :param lon1: Долгота точки А
        :param lat2: Широта точки Б
        :param lon2: Долгота точки Б
        :param n_digits: Округляем полученный ответ до n знака после запятой
        :return: Значение угла с точностью до n_digits
    """

    lat1, lon1 = np.radians(round(lat1, 6)), np.radians(round(lon1, 6))
    lat2, lon2 = np.radians(round(lat2, 6)), np.radians(round(lon2, 6))

    dlon = (lon2 - lon1)
    numerator = np.sin(dlon) * np.cos(lat2)
    denominator = np.cos(lat1) * np.sin(lat2) - (np.sin(lat1) * np.cos(lat2) * np.cos(dlon))

    theta = np.arctan2(numerator, denominator)
    theta_deg = (np.degrees(theta) + 360) % 360

    return round(theta_deg, n_digits)

In [16]:
train['bearing_degree_1'] = bearing_degree(train['lat'], train['lon'], train['merchant_lat'], train['merchant_lon'], ).values
test['bearing_degree_1'] = bearing_degree(test['lat'], test['lon'], test['merchant_lat'], test['merchant_lon'], ).values
train['bearing_degree_2'] = bearing_degree(train['lat'], train['lon'], 0, 0, ).values
test['bearing_degree_2'] = bearing_degree(test['lat'], test['lon'], 0, 0, ).values

train['bearing_degree_3'] = bearing_degree(0, 0, train['merchant_lat'], train['merchant_lon'], ).values
test['bearing_degree_3'] = bearing_degree(0, 0, test['merchant_lat'], test['merchant_lon'], ).values

train['hav_dist_1'] = haversine_distance(train['lat'], train['lon'], train['merchant_lat'], train['merchant_lon'], ).values
test['hav_dist_1'] = haversine_distance(test['lat'], test['lon'], test['merchant_lat'], test['merchant_lon'], ).values


train['hav_dist_2'] = haversine_distance(train['lat'], train['lon'], 0, 0, ).values
test['hav_dist_2'] = haversine_distance(test['lat'], test['lon'], 0, 0, ).values

train['hav_dist_3'] = haversine_distance(0, 0, train['merchant_lat'], train['merchant_lon'], ).values
test['hav_dist_3'] = haversine_distance(0, 0, test['merchant_lat'], test['merchant_lon'], ).values

### Логарифмизация и ночные транзакции, превышение порога (перцентиль уровня 0.9)

In [18]:
import numpy as np

# Логарифм суммы
train['amount_log'] = np.log1p(train['amount']) 
test['amount_log'] = np.log1p(test['amount'])


# Фильтруем ночные транзакции
train_night = train[train['transaction_time_binning_by_part'] == "night"]
test_night = test[test['transaction_time_binning_by_part'] == "night"]

# Агрегаты по ночным транзакциям для каждого дня
train_night_agg = train_night.groupby('transaction_date')['amount'].agg(
    sum_by_night_part='sum',
    mean_by_night_part='mean',
    median_by_night_part='median'
).reset_index()

test_night_agg = test_night.groupby('transaction_date')['amount'].agg(
    sum_by_night_part='sum',
    mean_by_night_part='mean',
    median_by_night_part='median'
).reset_index()

# Объединяем
train = train.merge(train_night_agg, on='transaction_date', how='left')
test = test.merge(test_night_agg, on='transaction_date', how='left')

# 90-й перцентиль для каждого дня
train_90 = train.groupby('transaction_date')['amount'].quantile(0.9).reset_index(name='percentile_90')
test_90 = test.groupby('transaction_date')['amount'].quantile(0.9).reset_index(name='percentile_90')

# Объединяем 90-й перцентиль с основными данными
train = train.merge(train_90, on='transaction_date', how='left')
test = test.merge(test_90, on='transaction_date', how='left')

# Сумма транзакций выше 90-го перцентиля по дням
train_high = train[train['amount'] > train['percentile_90']].groupby('transaction_date')['amount'].sum().reset_index(name='sum_more_than_90')
test_high = test[test['amount'] > test['percentile_90']].groupby('transaction_date')['amount'].sum().reset_index(name='sum_more_than_90')

# Объединяем с основными данными
train = train.merge(train_high, on='transaction_date', how='left')
test = test.merge(test_high, on='transaction_date', how='left')

# Заполняем пропущенные значения нулями (если в день не было ночных транзакций?)
train[['sum_by_night_part', 'mean_by_night_part', 'median_by_night_part', 'sum_more_than_90']] = \
    train[['sum_by_night_part', 'mean_by_night_part', 'median_by_night_part', 'sum_more_than_90']].fillna(0)

test[['sum_by_night_part', 'mean_by_night_part', 'median_by_night_part', 'sum_more_than_90']] = \
    test[['sum_by_night_part', 'mean_by_night_part', 'median_by_night_part', 'sum_more_than_90']].fillna(0)

### Статистики по полу

In [20]:
# Аггрегируем данные по дню и полу для train по транзакциям
gender_daily_stats_train = train.groupby(['transaction_date', 'gender']).agg(
    count=('amount', 'size'),           # Количество
    mean_amount=('amount', 'mean'),     # Среднее  
    median_amount=('amount', 'median'), # Медианное  
    total_sum=('amount', 'sum')        # Сумма 
).reset_index()

# Объединяем результаты с исходным train
train = pd.merge(train, gender_daily_stats_train, on=['transaction_date', 'gender'], how='left')

# Аггрегируем данные по дню и полу для test
gender_daily_stats_test = test.groupby(['transaction_date', 'gender']).agg(
    count=('amount', 'size'),           # Количество 
    mean_amount=('amount', 'mean'),     # Среднее  
    median_amount=('amount', 'median'), # Медианное  
    total_sum=('amount', 'sum')        # Сумма 
).reset_index()

# Объединяем результаты с исходным test
test = pd.merge(test, gender_daily_stats_test, on=['transaction_date', 'gender'], how='left')


### Подумать

#### Бинаризация (дискретизация) числовых признаков

Наконец, подумаем о предсказании стоимости квартиры по расстоянию до ближайшей станции метро $x_j$.

Может оказаться, что самые дорогие квартиры расположены где-то в 5-10 минутах ходьбы от метро, а те, что ближе или дальше, стоят не так дорого. В этом случае зависимость целевой переменной от признака не будет линейной. Чтобы сделать линейную модель подходящей, мы можем бинаризовать признак. Для этого выберем некоторую сетку точек ${t_1, \dots, t_m}$. Это может быть равномерная сетка между минимальным и максимальным значением признака или, например, сетка из эмпирических квантилей. Добавим сюда точки $t_0 = -\infty$ и $t_{m+1} = +\infty$.  

Новые признаки зададим как:
$$\large
b_i(x) = [t_{i-1} < x_j \leq t_i], \quad i = 1, \dots, m+1.
$$
Линейная модель над этими признаками будет выглядеть как:
$$\large
a(x) = w_1 [t_0 < x_j \leq t_1] + \dots + w_{m+1} [t_m < x_j \leq t_{m+1}] + \dots,
$$
то есть мы найдём свой прогноз стоимости квартиры для каждого интервала расстояния до метро. Такой подход позволит учесть нелинейную зависимость между признаком и целевой переменной.


### Подумать

#### DBSKAN / Optics
#### Isolation Forest

### CatBoost Encoding

In [28]:
train['ts_transaction_time'] = pd.to_datetime(train['transaction_time']).values.astype('int64') // 10**9
test['ts_transaction_time'] = pd.to_datetime(test['transaction_time']).values.astype('int64') // 10**9

In [29]:
# Преобразуем все столбцы в строки перед кодированием
cat_columns = ['gender', 'jobs', 'transaction_time_holidays', 'transaction_time_binning_by_part']
for col in cat_columns:
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)

train[cat_columns] = train[cat_columns].fillna('пропуск')
test[cat_columns] = test[cat_columns].fillna('пропуск')


numeric_features = [
    'amount',
    'lat',
    'lon',
    'population_city',
    'merchant_lat',
    'merchant_lon',
    'amount_log',
    'sum_by_night_part',
    'mean_by_night_part',
    'median_by_night_part',
    'percentile_90',
    'sum_more_than_90',
    'count',
    'mean_amount',
    'median_amount',
    'total_sum',
    'transaction_time_month',
    'transaction_time_week',
    'transaction_time_day_of_the_week',
    'transaction_time_hour',
    'transaction_time_minute',
    'bearing_degree_1',
    'bearing_degree_2',
    'bearing_degree_3',
    'hav_dist_1',
    'hav_dist_2',
    'hav_dist_3'
]

cat_columns = [
    'merch',
    'cat_id',
    'name_1',
    'name_2',
    'gender',  
    'street',
    'one_city',
    'us_state',
    'jobs',  
    'post_code',
    'transaction_time_holidays',  
    'transaction_time_binning_by_part' 
]

target_enc = ce.CatBoostEncoder(cols=cat_columns)
target_enc = target_enc.fit(train[cat_columns], train['target'])
train = train.join(target_enc.transform(train[cat_columns]).add_suffix('_cb'))
test = test.join(target_enc.transform(test[cat_columns]).add_suffix('_cb'))


In [30]:
train.columns

Index(['transaction_time', 'merch', 'cat_id', 'amount', 'name_1', 'name_2',
       'gender', 'street', 'one_city', 'us_state', 'post_code', 'lat', 'lon',
       'population_city', 'jobs', 'merchant_lat', 'merchant_lon', 'target',
       'transaction_date', 'transaction_time_month', 'transaction_time_week',
       'transaction_time_day_of_the_week', 'transaction_time_hour',
       'transaction_time_minute', 'transaction_time_holidays',
       'transaction_time_binning_by_part', 'bearing_degree_1',
       'bearing_degree_2', 'bearing_degree_3', 'hav_dist_1', 'hav_dist_2',
       'hav_dist_3', 'amount_log', 'sum_by_night_part', 'mean_by_night_part',
       'median_by_night_part', 'percentile_90', 'sum_more_than_90', 'count',
       'mean_amount', 'median_amount', 'total_sum', 'ts_transaction_time',
       'merch_cb', 'cat_id_cb', 'name_1_cb', 'name_2_cb', 'gender_cb',
       'street_cb', 'one_city_cb', 'us_state_cb', 'jobs_cb', 'post_code_cb',
       'transaction_time_holidays_cb', 'trans

In [31]:
model_features = [
    'amount', 
    'amount_log', 
    'population_city', 
    'lat', 
    'lon', 
    'merchant_lat', 
    'merchant_lon', 
    'bearing_degree_1',
    'bearing_degree_2', 
    'bearing_degree_3', 
    'hav_dist_1', 
    'hav_dist_2',
    'hav_dist_3',
    'transaction_time_month', 
    'transaction_time_week',
    'transaction_time_day_of_the_week', 
    'transaction_time_hour',
    'transaction_time_minute', 
    'sum_by_night_part', 
    'mean_by_night_part', 
    'median_by_night_part',
    'percentile_90', 
    'sum_more_than_90', 
    'count', 
    'mean_amount',
    'median_amount', 
    'total_sum',
    'merch_cb', 
    'cat_id_cb', 
    'name_1_cb', 
    'name_2_cb',
    'gender_cb', 
    'street_cb', 
    'one_city_cb',
    'us_state_cb', 
    'jobs_cb',
    'post_code_cb',
    'transaction_time_holidays_cb',
    'transaction_time_binning_by_part_cb'
]

## Baseline Model из лекции

In [33]:
# автоподбор трешхолда из sklearn

In [34]:
# model = DecisionTreeClassifier(max_depth=5, random_state=15, )
model = LGBMClassifier(n_estimators=50, random_state=14, )

X_train, X_test, y_train, y_test = train_test_split(
    train[model_features], train['target'], test_size=0.25, random_state=42,
    stratify=train['target']
)


%time
model = model.fit(X_train, y_train)

predictions = model.predict_proba(X_test)

predictions = predictions[:, 1]

# TODO: подобрать threshold модели (поменять np.mean(predictions))
predictions_binary = predictions >= np.mean(predictions)

f1_score(y_test, predictions_binary)


# ВНИМАНИЕ: f1 = 0.3692127368626685 без фича инжиниринга

# ДВОЙНОЕ ВНИМАНИЕ: f1 = 0.5428809325562032 с фича инжинирингом. даже без трешхолда

CPU times: total: 0 ns
Wall time: 0 ns


D:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "D:\Anaconda\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


[LightGBM] [Info] Number of positive: 3378, number of negative: 586445
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036679 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 7111
[LightGBM] [Info] Number of data points in the train set: 589823, number of used features: 39
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.005727 -> initscore=-5.156795
[LightGBM] [Info] Start training from score -5.156795


0.5428809325562032

## CatBoost

In [36]:
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(
    train[model_features], train['target'], test_size=0.25, random_state=42,
    stratify=train['target']
)

# Обучение модели
model_catboost = CatBoostClassifier(iterations=50, random_state=14, verbose=0)
model_catboost.fit(X_train, y_train)

# Получение вероятностей на валидации
predictions_proba_val = model_catboost.predict_proba(X_val)[:, 1]

# Функция для поиска оптимального порога
def find_best_threshold(y_true, predictions):
    best_threshold, best_f1 = 0, 0
    for threshold in np.arange(0, 1, 0.01):
        f1 = f1_score(y_true, predictions >= threshold)
        if f1 > best_f1:
            best_f1, best_threshold = f1, threshold
    return best_threshold, best_f1

# Оптимальный порог по валидационной выборке
best_threshold, best_f1 = find_best_threshold(y_val, predictions_proba_val)
print(f"Оптимальный порог: {best_threshold}")
print(f"F1-score: {best_f1}")

# Оптимальный порог: 0.46
# F1-score: 0.8356940509915014


Оптимальный порог: 0.46
F1-score: 0.8356940509915014


### CatBoost с затюненными гиперпараметрами

Ниже код для Optuna (в конце ноутбука)

In [39]:
# Оптимальные параметры
best_params = {
    'iterations': 200,
    'learning_rate': 0.24008436863847432,
    'depth': 10,
    'l2_leaf_reg': 5.064713404918217,
    'border_count': 128
}

# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(
    train[model_features], train['target'], test_size=0.25, random_state=42,
    stratify=train['target']
)

# Обучение модели с оптимальными параметрами - сразу передаю их
model_catboost = CatBoostClassifier(
    iterations=best_params['iterations'],
    learning_rate=best_params['learning_rate'],
    depth=best_params['depth'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    border_count=best_params['border_count'],
    random_state=14,
    verbose=0
)
model_catboost.fit(X_train, y_train)

# Получение вероятностей на валидации
predictions_proba_val = model_catboost.predict_proba(X_val)[:, 1]

# Функция для поиска оптимального порога - сделал по-простому
def find_best_threshold(y_true, predictions):
    best_threshold, best_f1 = 0, 0
    for threshold in np.arange(0, 1, 0.01):
        f1 = f1_score(y_true, predictions >= threshold)
        if f1 > best_f1:
            best_f1, best_threshold = f1, threshold
    return best_threshold, best_f1

# Оптимальный порог по валидационной выборке
best_threshold, best_f1 = find_best_threshold(y_val, predictions_proba_val)
print(f"Оптимальный порог: {best_threshold}")
print(f"Лучший F1-score: {best_f1}")

# Применяем оптимальный порог
predictions_val = predictions_proba_val >= best_threshold

# Получаем F1-score на валидационной выборке (оптимальный порог)
f1_val = f1_score(y_val, predictions_val)
print(f"F1-score на валидации с оптимальным порогом: {f1_val}")


# Оптимальный порог: 0.29
# Лучший F1-score: 0.8554437328453797
# F1-score на валидации с оптимальным порогом: 0.8554437328453797

Оптимальный порог: 0.29
Лучший F1-score: 0.8554437328453797
F1-score на валидации с оптимальным порогом: 0.8554437328453797


In [40]:
# Предсказания для тестовых данных
test_preds = model_catboost.predict_proba(test[model_features])[:, 1]
test_predictions = test_preds >= best_threshold

predictions_df = pd.DataFrame({
    'index': test.index,
    'prediction': test_predictions
})

predictions_df.to_csv('predictions.csv', index=False)

In [41]:
# Предсказания для тестового датасета
predictions_proba_test = model_catboost.predict_proba(test[model_features])[:, 1]
predictions_binary_test = (predictions_proba_test >= best_threshold).astype(int)

results_test = pd.DataFrame({
    'index': test.index,  
    'prediction': predictions_binary_test
})

results_test.to_excel('submission.xlsx', index=False)


#### Гиперпараметры

**Hyperopt**

В библиотеке [Hyperopt](http://hyperopt.github.io/hyperopt/) реализованы три метода оптимизации гиперпараметров:

- Random Search
- TPE
- [Adaptive TPE](https://github.com/electricbrainio/hypermax)

У них есть небольшой [туториал](https://github.com/hyperopt/hyperopt/wiki/FMin) по тому, как начать пользоваться библиотекой. Кроме того, у них есть обёртка над sklearn, позволяющая работать с моделями оттуда: [Hyperopt-sklearn](https://github.com/hyperopt/hyperopt-sklearn).

**Optuna**

В библиотеке [Optuna](https://optuna.org/) реализованы те же методы оптимизации, что и в Hyperopt, но по многим параметрам она оказывается удобнее. Хорошее сравнение Optuna и Hyperopt можно найти [здесь](https://neptune.ai/blog/optuna-vs-hyperopt).

In [43]:
# Лучшие параметры:
# {'iterations': 200, 'learning_rate': 0.24008436863847432, 'depth': 10, 'l2_leaf_reg': 5.064713404918217, 'border_count': 128}
# Лучший F1-score: 0.8554437328453797

In [44]:
# Разделение на обучающую и тестовую выборки
X_train, X_val, y_train, y_val = train_test_split(
    train[model_features], train['target'], test_size=0.25, random_state=42,
    stratify=train['target']
)

# Функция для поиска оптимального порога
def find_best_threshold(y_true, predictions):
    best_threshold, best_f1 = 0, 0
    for threshold in np.arange(0, 1, 0.01):
        f1 = f1_score(y_true, predictions >= threshold)
        if f1 > best_f1:
            best_f1, best_threshold = f1, threshold
    return best_threshold, best_f1

# Определяем целевую функцию для Optuna
def objective(trial):
    # Определяем параметры
    iterations = trial.suggest_int('iterations', 50, 200, step=50)  # Количество итераций
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)  # Скорость обучения
    depth = trial.suggest_int('depth', 3, 10)  # Глубина дерева
    l2_leaf_reg = trial.suggest_float('l2_leaf_reg', 1, 10)  # Регуляризация
    border_count = trial.suggest_int('border_count', 32, 255, step=32)  # Количество границ

    # Создаем модель с гиперпараметрами по Optuna
    model_catboost = CatBoostClassifier(
        iterations=iterations,
        learning_rate=learning_rate,
        depth=depth,
        l2_leaf_reg=l2_leaf_reg,
        border_count=border_count,
        random_state=14,
        verbose=0
    )
    
    # Обучаем
    model_catboost.fit(X_train, y_train)
    
    # Получаем предсказания на валидации
    predictions_proba_val = model_catboost.predict_proba(X_val)[:, 1]
    
    # Оптимальный порог по валидационной выборке
    best_threshold, best_f1 = find_best_threshold(y_val, predictions_proba_val)
    
    return best_f1 

# study
study = optuna.create_study(direction='maximize')  # Максимизация F1-score

# Запуск оптимизации
study.optimize(objective, n_trials=30)  # Проводим 30 испытаний

# Выводим результаты
print("Лучшие параметры:")
print(study.best_params)
print("Лучший F1-score:", study.best_value)